In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pyfinance.ols import PandasRollingOLS
import re
from dateutil.relativedelta import *

import warnings
warnings.filterwarnings("ignore")

数据处理

In [2]:
#读取全品种历史数据
df = pd.read_excel('.//B.1 - Choice数据//全品种期货主连数据.xlsx')

#筛选出目标品种
PZ_list = pd.read_excel('.//B.1 - Choice数据//future-factor_Choice版.xlsx')
PZ_list =  PZ_list['品种'].values.tolist()
df = df[df['品种'].isin(PZ_list)]
df

,时间,代码,对应主力,收盘价,前收盘价,收盘涨跌幅,结算价,前结算价,结算涨跌幅,成交量,成交金额,持仓量,持仓量变化,品种,板块
0,2013-12-31,A0.DCE,A1405.DCE,4441.0,4411.0,0.006801,4434.0,4412.0,0.004986,60974.0,2.704132e+09,66464.0,-4306.0,A,农产品
1,2014-01-02,A0.DCE,A1409.DCE,4322.0,4321.0,0.000231,4311.0,4312.0,-0.000232,27710.0,1.194784e+09,79750.0,13286.0,A,农产品
2,2014-01-03,A0.DCE,A1409.DCE,4326.0,4322.0,0.000925,4316.0,4311.0,0.001160,18792.0,8.111754e+08,77098.0,-2652.0,A,农产品
3,2014-01-06,A0.DCE,A1409.DCE,4334.0,4326.0,0.001849,4331.0,4316.0,0.003475,22442.0,9.721491e+08,75212.0,-1886.0,A,农产品
4,2014-01-07,A0.DCE,A1409.DCE,4338.0,4334.0,0.000923,4336.0,4331.0,0.001154,23942.0,1.038202e+09,68690.0,-6522.0,A,农产品
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140347,2022-12-26,ZN0.SHF,ZN2302.SHF,23520.0,23155.0,0.015763,23395.0,23145.0,0.010801,120987.0,1.415501e+10,91808.0,51860.0,ZN,有色金属
140348,2022-12-27,ZN0.SHF,ZN2302.SHF,23810.0,23520.0,0.012330,23660.0,23395.0,0.011327,91589.0,1.083679e+10,92097.0,289.0,ZN,有色金属
140349,2022-12-28,ZN0.SHF,ZN2302.SHF,23965.0,23810.0,0.006510,23805.0,23660.0,0.006128,97112.0,1.156045e+10,90981.0,-1116.0,ZN,有色金属
140350,2022-12-29,ZN0.SHF,ZN2302.SHF,23800.0,23965.0,-0.006885,23840.0,23805.0,0.001470,69835.0,8.325043e+09,88973.0,-2008.0,ZN,有色金属


In [3]:
#剔除上市半年内（125个交易日）的合约
for c in PZ_list:
    if df.loc[df['品种']==c]['时间'].astype(str).values[0] != '2013-12-31':
        df = df.drop(df.loc[df['品种']==c].index[:125])
    else:
        pass
    
df = df.reset_index(drop=True)

In [4]:
#计算平均成交量，按需剔除流动性弱的品种
df['平均成交'] = 0
for i in PZ_list:
    ave_vol = df.loc[df['品种']==i]['成交量'].shift().rolling(20).mean()
    df.loc[(df['品种']==i), '平均成交'] = ave_vol

In [5]:
#用于计算持仓金额
multi = pd.read_excel('./B.1 - Choice数据/主连数据_choice.xlsx',sheet_name='合约乘数')
multi = multi.iloc[2:,np.r_[0,2]]
multi.columns = ['主连品种','合约乘数']
multi = multi.dropna()
multi = multi.reset_index(drop=True)
multi['品种'] = 0
for i in range(len(multi)):
    multi['品种'][i] = re.findall('[a-zA-Z]+',multi['主连品种'][i])[0]

定义因子计算函数

In [6]:
def multisum(x):   # 定义连加函数,返回连加后的最后一个值
    
    summ = x.cumsum()
    summ = summ.iloc[-1]
    
    return summ    

def multiprod(x):   # 定义连乘函数，返回连乘后的最后一个值
    
    prod = x.cumprod()
    prod = prod.iloc[-1]
    
    return prod

In [7]:
def LongShort(table,ret,position):
    """计算Long-Short组合收益率
    
    Parameters:
     table -  已包含多空方向的dataframe  
     ret - 收盘涨跌幅 or 结算涨跌幅
     position - ts_position or cs_position 指定时序动量还是截面多空类因子
    
    Returns：
     包含多头组合retn，空头组合ret和多空组合ret的dataframe

    """
   
    #多头组合
    table_long = table.loc[table[position] == 1]  
    ret_long = pd.DataFrame(table_long.groupby(by='时间')[ret].mean()) 

    #空头组合
    table_short = table.loc[table[position] == -1]
    ret_short = pd.DataFrame(table_short.groupby(by='时间')[ret].mean())

    #时序动量因子
    long_short = pd.merge(ret_long,ret_short,how='outer',on='时间').fillna(0).sort_index()
    long_short.columns = ['long_return','short_return']
    long_short['factor_return'] = long_short['long_return'] - long_short['short_return']

    return long_short

In [8]:
def cs_position(table):
    """标记截面类策略多、空头
    
    Parameters:
     table - 已包含Rank的dataframe
    
    Returns:
     包含多空方向的dataframe
     
    """
    
    time = set(table['时间'])
    time = list(time)
    time.sort()
    
    #截面动量仓位
    table['cs_position'] = 0
    for day in time:
        n = len(table.loc[(table['时间']==day), :])
        select = n * 0.2
        if (select > 0.2) & (select<1):
            select = 1
        else:
            select = round(n * 0.2)
        table.loc[(table['时间']==day) & (table['rank']<= select), 'cs_position'] = 1
        table.loc[(table['时间']==day) & (table['rank']> n-select), 'cs_position'] = -1
        
    return table    

In [9]:
class TimeSeries_ave(object):  
    """时序动量，所有品种收益直接平均
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
        
    def ts_mom(self):      #时序动量
        
        table = df.copy()
        
        table['ts_mom'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'ts_mom'] = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(multisum)
        table = table.dropna()
        
        table['ts_position'] = np.sign(table['ts_mom'])
        
        #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #直接平均
        table['factor_return'] = table[ret]*table['ts_position']
        
        return table

In [10]:
class TimeSeries_ls(object): 
    """时序动量，品种收益多头减空头
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
        
    def ts_mom(self):      #时序动量
        
        table = df.copy()
        
        table['ts_mom'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'ts_mom'] = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(multisum)
        table = table.dropna()
        
        table['ts_position'] = np.sign(table['ts_mom'])
        
        #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #多空相减
        long_short = LongShort(table,self.ret,'ts_position')
#         table['factor_return'] = long_short['factor_return']
        
        return long_short

In [11]:
class TimeSeries_ls_bk(object):
    """【板块】时序动量，所有品种收益多头减空头
    
    Attributes:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
        
    def ts_mom(self):      #时序动量
        """计算时序动量

        Returns:
         A dataframe, columns是板块因子净值

        """

        table = df.copy()
        bk = set(table['板块'])
        bk = list(bk)
        
        table['ts_mom'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'ts_mom'] = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(multisum)
        table = table.dropna()
        
        table['ts_position'] = np.sign(table['ts_mom'])
        
        #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #多空相减
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[df['板块'] == item]
            long_short = LongShort(df_bk,self.ret,'ts_position')
            net_values[item] = NetValues(long_short)['net_value']
            
        return net_values

In [12]:
class CrossSection_ave(object):
    """截面类因子，所有品种收益直接平均
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
    
    def cs_mom(self):
        """计算截面动量因子
        
        """
        ts_factor = TimeSeries_ave(self.R,self.L,self.ret,self.limit)
        table = ts_factor.ts_mom().copy()
        
        #时序动量在截面上排序
        table['rank'] = table.groupby(by='时间')['ts_mom'].rank(axis=0, method='dense', ascending=False)
        
        table = cs_position(table)
        table['factor_return'] = table[self.ret]*table['cs_position']
        
        return table
    
    def skew(self):           
        """计算偏度因子
        
        """
        table = df.copy()
        
        table['skew'] = 0
        for j in PZ_list:
            factor = table.loc[table['品种']==j][self.ret].shift(L+1).rolling(self.R).apply(lambda x:x.skew())     #无偏估计
        #     factor = table.loc[table['品种']==j]['lnret'].shift(L+1).rolling(R).apply(lambda x:stats.skew(x))   #有偏估计
            table.loc[(table['品种']==j), 'skew'] = factor
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        table['rank'] = table.groupby(by='时间')['skew'].rank(axis=0, method='dense', ascending=True)
        
        table = cs_position(table)
        table['factor_return'] = table[self.ret]*table['cs_position']
        
        return table
    
    def liquid(self): 
        """计算流动性因子
        
        """
        table = df.copy()
        
        #计算流动性因子
        # table['Amivest'] = table['成交金额']/(abs(table['mod_lnret']) * 1e+8 + 1e-8)     #调整量级
        table['Amivest'] = abs(table[self.ret])/table['成交量'] 
        for j in PZ_list:
            factor = table.loc[table['品种']==j]['Amivest'].shift(self.L+1).rolling(self.R).mean()     
            table.loc[(table['品种']==j), 'liquid'] = factor
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #流动性因子在截面上排序
        table['rank'] = table.groupby(by='时间')['liquid'].rank(axis=0, method='dense', ascending=False)
        
        table = cs_position(table)
        table['factor_return'] = table[self.ret]*table['cs_position']
        
        return table
    
    def ma(self):
        """计算均价突破因子
        
        """
        table = df.copy()
        
        #计算均价MA
        for j in PZ_list:
            table.loc[(table['品种']==j), 'price_ave'] = table.loc[table['品种']==j][self.ret[:2]+'价'].shift(self.L+1).rolling(self.R).mean()
            table.loc[(table['品种']==j), 'price_t-20'] = table.loc[table['品种']==j][self.ret[:2]+'价'].shift(self.R) 
            table.loc[(table['品种']==j), 'sigma'] = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).std() 

        #均价突破因子
        table['MA'] = (table[self.ret[:2]+'价']-table['price_ave'])/table['sigma']/table['price_t-20']
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #均价突破因子在截面上排序
        table['rank'] = table.groupby(by='时间')['MA'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        table['factor_return'] = table[self.ret]*table['cs_position']
        
        return table
    
    def delta_oi(self):
        """计算持仓变化因子
        
        """
        table = df.copy()
        table = pd.merge(table,multi,on='品种',how='left')
        
        #计算持仓变化因子
        table['持仓金额'] = table['持仓量']*table['合约乘数']*table['结算价']
        for j in PZ_list:
            table.loc[(table['品种']==j), 'deltaOI'] = (table.loc[table['品种']==j]['持仓金额'] - table.loc[table['品种']==j]['持仓金额'].shift(R)).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        table['rank'] = table.groupby(by='时间')['deltaOI'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        table['factor_return'] = table[self.ret]*table['cs_position']
        
        return table

In [62]:
class CrossSection_ls(object): 
    """截面类因子，品种收益多头减空头
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
            
    def cs_mom(self):     
        """计算截面动量因子
        
        """
        ts_factor = TimeSeries_ave(self.R,self.L,self.ret,self.limit)
        table = ts_factor.ts_mom().copy()
        
        #时序动量在截面上排序
        table['rank'] = table.groupby(by='时间')['ts_mom'].rank(axis=0, method='dense', ascending=False)
        
#         long_short = cs_position(table)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        return long_short
    
    def skew(self):      
        """计算偏度因子
        
        """
        table = df.copy()
        
        table['skew'] = 0
        for j in PZ_list:
            factor = table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(lambda x:x.skew())     #无偏估计
        #     factor = table.loc[table['品种']==j]['lnret'].shift(L+1).rolling(R).apply(lambda x:stats.skew(x))   #有偏估计
            table.loc[(table['品种']==j), 'skew'] = factor
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        table['rank'] = table.groupby(by='时间')['skew'].rank(axis=0, method='dense', ascending=True)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return long_short
    
    def cv(self): 
        """计算波动因子
        
        """
        table = df.copy()
        
        table['variance'] = 0
        table['avg_ret'] = 0
        table['CV'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'variance'] =  table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(lambda x:np.var(x))
            table.loc[(table['品种']==j), 'avg_ret'] =  table.loc[table['品种']==j][self.ret].shift(self.L+1).rolling(self.R).apply(lambda x:np.mean(x))
            table.loc[(table['品种']==j), 'CV'] = table.loc[(table['品种']==j), 'variance']/abs(table.loc[(table['品种']==j), 'avg_ret'])
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #变异系数在截面上排序
        table['rank'] = table.groupby(by='时间')['CV'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return long_short
    
    def liquid(self):
        """计算流动性因子
        
        """
        table = df.copy()
        
        #计算流动性因子
        table['Amivest'] = abs(table[self.ret])/table['成交金额'] 
        for j in PZ_list:
            factor = table.loc[table['品种']==j]['Amivest'].rolling(self.R).mean().shift(self.L+1)     
            table.loc[(table['品种']==j), 'liquid'] = factor
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #流动性因子在截面上排序
        table['rank'] = table.groupby(by='时间')['liquid'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return long_short
        
    def ma(self):
        """计算均价突破因子
        
        """
        table = df.copy()

        #计算均价MA
        for j in PZ_list:
            table.loc[(table['品种']==j), 'price_ave'] = table.loc[table['品种']==j][self.ret[:2]+'价'].rolling(self.R).mean()
            table.loc[(table['品种']==j), 'price_t-20'] = table.loc[table['品种']==j][self.ret[:2]+'价'].shift(self.R-1) 
            table.loc[(table['品种']==j), 'sigma'] = table.loc[table['品种']==j][self.ret].rolling(self.R).std() 

        #均价突破因子
        table['MA'] = ((table[self.ret[:2]+'价']-table['price_ave'])/table['sigma']/table['price_t-20']).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #均价突破因子在截面上排序
        table['rank'] = table.groupby(by='时间')['MA'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return long_short

    def delta_oi(self):
        """计算持仓变化因子
        
        """
        table = df.copy()
        table = pd.merge(table,multi,on='品种',how='left')
        
        #计算持仓变化因子
        table['持仓金额'] = table['持仓量']*table['合约乘数']*table['结算价']
        for j in PZ_list:
            table.loc[(table['品种']==j), 'deltaOI'] = (table.loc[table['品种']==j]['持仓金额'] - table.loc[table['品种']==j]['持仓金额'].shift(R)).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        table['rank'] = table.groupby(by='时间')['deltaOI'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,self.ret,'cs_position')
        
        return long_short
    
    def beta(self):
        """计算beta因子
        
        """
        table = df.copy()

        table['year'] = table['时间'].dt.year
        table['month'] = table['时间'].dt.month
        
        #合并beta因子
        betas = pd.read_excel('./A.7 - Beta因子/beta系数.xlsx')
        betas['lag时间'] = 0
        for z in range(len(betas)):
            betas.loc[z,'lag时间'] = betas.loc[z,'时间'] + relativedelta(months = +1)
        betas['lag时间'] = pd.to_datetime(betas['lag时间'])
        betas['year'] = betas['lag时间'].dt.year
        betas['month'] = betas['lag时间'].dt.month
        del betas['时间']
        del betas['lag时间']
        table = pd.merge(table,betas,how='left',on=['品种','year','month'])
        table = table[table['beta'].notna()]
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #beta因子在截面上排序
        table['rank'] = table.groupby(by='时间')['beta'].rank(axis=0, method='dense', ascending=False)
        table = cs_position(table)
        long_short =  LongShort(table,ret,'cs_position')
        
        return long_short

In [14]:
class CrossSection_ls_bk(object):
    """【板块】截面类因子，品种收益多头减空头
    
    Parameters:
     R - retroactive period
     L - lag period
     ret - 收盘涨跌幅 or 结算涨跌幅
     limit - 成交量阈值，剔除低于该值的品种
    
    """

    def __init__(self,R,L,ret,limit):

        self.R = R
        self.L = L
        self.ret = ret
        self.limit = limit
            
    def cs_mom(self):
        """计算板块截面动量
        
        """
        ts_factor = TimeSeries_ave(R,L,ret,limit)    #用ave方法原因：ave方法返回的是table，ls方法返回的只是return
        table = ts_factor.ts_mom().copy()
        bk = set(table['板块'])
        bk = list(bk)
        
        #时序动量在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['ts_mom'].rank(axis=0, method='dense', ascending=False)
        
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']

        return net_values
    
    def skew(self):       
        """计算板块偏度因子
        
        """
        table = df.copy()
        bk = set(table['板块'])
        bk = list(bk)
        
        table['skew'] = 0
        for j in PZ_list:
            factor = table.loc[table['品种']==j][self.ret].shift(L+1).rolling(R).apply(lambda x:x.skew())     #无偏估计
        #     factor = table.loc[table['品种']==j]['lnret'].shift(L+1).rolling(R).apply(lambda x:stats.skew(x))   #有偏估计
            table.loc[(table['品种']==j), 'skew'] = factor
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['skew'].rank(axis=0, method='dense', ascending=True)
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']
        
        return net_values
    
    def cv(self):                   
        """计算板块波动因子
        
        """
        table = df.copy()
        bk = set(table['板块'])
        bk = list(bk)
        
        table['variance'] = 0
        table['avg_ret'] = 0
        table['CV'] = 0
        for j in PZ_list:
            table.loc[(table['品种']==j), 'variance'] =  table.loc[table['品种']==j][ret].shift(L+1).rolling(R).apply(lambda x:np.var(x))
            table.loc[(table['品种']==j), 'avg_ret'] =  table.loc[table['品种']==j][ret].shift(L+1).rolling(R).apply(lambda x:np.mean(x))
            table.loc[(table['品种']==j), 'CV'] = table.loc[(table['品种']==j), 'variance']/abs(table.loc[(table['品种']==j), 'avg_ret'])
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #变异系数在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['CV'].rank(axis=0, method='dense', ascending=False)
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']
        
        return net_values
    
    def liquid(self):
        """计算板块流动性因子
        
        """
        table = df.copy()
        bk = set(table['板块'])
        bk = list(bk)
        
        #计算流动性因子
        table['Amivest'] = abs(table[ret])/table['成交金额'] 
        for j in PZ_list:
            factor = table.loc[table['品种']==j]['Amivest'].rolling(R).mean().shift(L+1)     
            table.loc[(table['品种']==j), 'liquid'] = factor
        table = table.dropna() 
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #流动性因子在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['liquid'].rank(axis=0, method='dense', ascending=False)
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']
        
        return net_values
        
    def ma(self):
        """计算板块均价突破因子
        
        """
        table = df.copy()
        bk = set(table['板块'])
        bk = list(bk)

        #计算均价MA
        for j in PZ_list:
            table.loc[(table['品种']==j), 'price_ave'] = table.loc[table['品种']==j][ret[:2]+'价'].rolling(R).mean()
            table.loc[(table['品种']==j), 'price_t-20'] = table.loc[table['品种']==j][ret[:2]+'价'].shift(R-1) 
            table.loc[(table['品种']==j), 'sigma'] = table.loc[table['品种']==j][ret].rolling(R).std() 

        #均价突破因子
        table['MA'] = ((table[ret[:2]+'价']-table['price_ave'])/table['sigma']/table['price_t-20']).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #均价突破因子在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['MA'].rank(axis=0, method='dense', ascending=False)
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']
        
        return lnet_values

    def delta_oi(self):
        """计算板块持仓变化因子
        
        """
        table = df.copy()
        table = pd.merge(table,multi,on='品种',how='left')
        bk = set(table['板块'])
        bk = list(bk)
        
        #计算持仓变化因子
        table['持仓金额'] = table['持仓量']*table['合约乘数']*table['结算价']
        for j in PZ_list:
            table.loc[(table['品种']==j), 'deltaOI'] = (table.loc[table['品种']==j]['持仓金额'] - table.loc[table['品种']==j]['持仓金额'].shift(R)).shift(1)
        table = table.dropna()
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #持仓变动因子在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['deltaOI'].rank(axis=0, method='dense', ascending=False)
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']
        
        return net_values
    
    def beta(self):
        """计算板块beta因子
        
        """
        table = df.copy()
        bk = set(table['板块'])
        bk = list(bk)

        table['year'] = table['时间'].dt.year
        table['month'] = table['时间'].dt.month
        
        #合并beta因子
        betas = pd.read_excel('./A.7 - Beta因子/beta系数.xlsx')
        betas['lag时间'] = 0
        for z in range(len(betas)):
            betas.loc[z,'lag时间'] = betas.loc[z,'时间'] + relativedelta(months = +1)
        betas['lag时间'] = pd.to_datetime(betas['lag时间'])
        betas['year'] = betas['lag时间'].dt.year
        betas['month'] = betas['lag时间'].dt.month
        del betas['时间']
        del betas['lag时间']
        table = pd.merge(table,betas,how='left',on=['品种','year','month'])
        table = table[table['beta'].notna()]
        
        # #按绝对平均成交量剔除
        table = table.loc[table['平均成交']>=self.limit]
        
        #beta因子在截面上排序
        net_values = pd.DataFrame(columns=bk)
        for item in bk:
            df_bk = table.loc[table['板块'] == item]
            df_bk['rank'] = df_bk.groupby(by='时间')['beta'].rank(axis=0, method='dense', ascending=False)
            df_bk = cs_position(df_bk)
            long_short =  LongShort(df_bk,ret,'cs_position')
            net_values[item] = NetValues(long_short)['net_value']
        
        return long_short

In [15]:
def NetValues(df):
    """计算因子净值序列
    
    Parameters:
     df - 包含因子return的dataframe
     
    Returns:
     包含因子虚拟净值序列的dataframe
    """
    
    net_values = pd.DataFrame(df.groupby(by='时间')['factor_return'].mean())
    net_values['1+ret'] = 1 + net_values['factor_return']
    net_values['net_value'] = net_values['1+ret'].cumprod()
    
    return net_values

单个因子计算

时序动量

In [46]:
R = 5 
L = 0
limit = 0
ret = '收盘涨跌幅'

In [17]:
ts_factor = TimeSeries_ave(R,L,ret,limit)   #实例化，直接平均

In [47]:
ts_factor = TimeSeries_ls(R,L,ret,limit)    #实例化，多头减空头

In [48]:
ts_mom = ts_factor.ts_mom()          

In [25]:
NetValues(ts_mom)

,factor_return,1+ret,net_value
时间,,,
2014-01-29,0.000044,1.000044,1.000044
2014-01-30,0.003932,1.003932,1.003977
2014-02-07,-0.004008,0.995992,0.999953
2014-02-10,-0.004768,0.995232,0.995184
2014-02-11,0.003015,1.003015,0.998185
...,...,...,...
2022-12-26,-0.003236,0.996764,1.224440
2022-12-27,0.001262,1.001262,1.225986
2022-12-28,-0.004514,0.995486,1.220452


In [26]:
a = NetValues(ts_mom)

In [27]:
a.to_excel('.//A.1 - 动量因子//时序动量因子_R=%s收盘多空头.xlsx'%R)

分板块

In [20]:
ts_factor_bk = TimeSeries_ls_bk(R,L,ret,limit)

In [23]:
ts_mom_bk = ts_factor_bk.ts_mom()

In [24]:
ts_mom_bk

,农产品,软商品,能源化工,油脂,黑色,有色金属,贵金属
时间,,,,,,,
2014-01-29,1.016693,0.994515,1.003030,0.997213,0.994830,0.999713,1.002934
2014-01-30,1.011233,0.997306,1.007986,1.002805,0.998315,1.001073,1.010447
2014-02-07,1.015482,0.983008,1.006768,0.973085,1.013768,1.006002,1.004795
2014-02-10,1.006470,0.992487,1.000256,0.971189,1.006786,1.003570,1.000682
2014-02-11,1.001797,0.987999,1.006293,0.961483,1.017311,1.000234,1.002342
...,...,...,...,...,...,...,...
2022-10-25,0.352795,0.560764,0.389817,0.933375,0.418521,0.396872,0.922630
2022-10-26,0.352081,0.562781,0.384489,0.919185,0.415752,0.396900,0.926631
2022-10-27,0.352845,0.566856,0.385390,0.916203,0.415694,0.396473,0.920229


In [25]:
ts_mom_bk.to_excel('.//A.1 - 动量因子//分板块时序动量因子_R=%s.xlsx'%R)

截面多空类因子

In [63]:
R = 5
L = 0
limit = 0
ret = '收盘涨跌幅'
method = 1            # 1-多空头， 2-直接平均

In [64]:
if method == 1:             #实例化
    cs_factor = CrossSection_ls(R,L,ret,limit)
else:
    cs_factor = CrossSection_ave(R,L,ret,limit)

In [65]:
cs_mom = cs_factor.cs_mom()       

In [66]:
NetValues(cs_mom)

,factor_return,1+ret,net_value
时间,,,
2014-01-29,0.007709,1.007709,1.007709
2014-01-30,0.006241,1.006241,1.013999
2014-02-07,-0.015633,0.984367,0.998147
2014-02-10,-0.003658,0.996342,0.994496
2014-02-11,0.002659,1.002659,0.997141
...,...,...,...
2022-12-26,-0.007042,0.992958,0.699179
2022-12-27,0.004990,1.004990,0.702668
2022-12-28,-0.000413,0.999587,0.702378


In [68]:
b = NetValues(cs_mom)

In [70]:
b.to_excel('.//A.1 - 动量因子//截面动量因子1_R=%s收盘多空头.xlsx'%R)

In [70]:
b.to_excel('.//A.2 - 偏度因子//偏度因子_R=%s收盘多空头.xlsx'%R)

In [94]:
b.to_excel('.//A.3 - 波动因子//波动因子_R=%s收盘多空头.xlsx'%R)

In [100]:
b.to_excel('.//A.4 - 流动性因子//流动性因子_R=%s收盘多空头成交金额.xlsx'%R)

In [106]:
b.to_excel('.//A.5 - 均价突破因子//均价突破因子_R=%s收盘多空头.xlsx'%R)

In [112]:
b.to_excel('.//A.6 - 持仓变化因子//持仓变化因子_R=%s收盘多空头.xlsx'%R)

In [125]:
b.to_excel('./A.7 - Beta因子/beta因子_36个月.xlsx')

分板块

In [40]:
cs_factor_bk = CrossSection_ls_bk(R,L,ret,limit)

In [41]:
skew_bk = cs_factor_bk.skew()

In [42]:
skew_bk

,能源化工,油脂,贵金属,软商品,黑色,有色金属,农产品
时间,,,,,,,
2015-01-09,0.948062,1.000342,0.999314,0.996695,1.004169,0.990376,1.011522
2015-01-12,0.898768,0.999412,1.001647,0.977743,0.997498,0.989077,1.006919
2015-01-13,0.885551,0.998799,1.000765,0.971039,0.998748,0.978233,1.016785
2015-01-14,0.910480,1.003636,1.002972,0.972618,0.994890,0.971612,1.036506
2015-01-15,0.906418,0.992272,0.997896,0.980306,1.010791,0.948305,1.028773
...,...,...,...,...,...,...,...
2022-10-25,0.403646,1.763735,1.655352,0.775350,1.112739,0.645888,0.656146
2022-10-26,0.400869,1.778467,1.639315,0.775640,1.115041,0.654466,0.655963
2022-10-27,0.399541,1.756105,1.654302,0.769177,1.121484,0.642962,0.652233


In [22]:
cs_mom_bk.to_excel('.//A.1 - 动量因子//分板块截面动量因子_R=%s.xlsx'%R)

In [43]:
skew_bk.to_excel('.//A.2 - 偏度因子//分板块偏度因子_R=%s.xlsx'%R)

In [38]:
cv_bk.to_excel('.//A.3 - 波动因子//分板块波动因子_R=%s.xlsx'%R)

In [ ]:
liquid_bk.to_excel('.//A.4 - 流动性因子//分板块流动性因子_R=%s.xlsx'%R) 

In [ ]:
ma_bk.to_excel('.//A.5 - 均价突破因子//分板块均价突破因子_R=%s.xlsx'%R)

In [ ]:
delta_oi_bk.to_excel('.//A.6 - 持仓变化因子//分板块持仓变化因子_R=%s.xlsx'%R)

In [ ]:
beta_bk.to_excel('./A.7 - Beta因子/分板块beta因子_36个月.xlsx')

月收益率与delta_CPI OLS回归（每月更新）

In [109]:
delta_CPI = pd.read_excel('./B.1 - Choice数据/同比CPI.xlsx')
# delta_CPI = delta_CPI.set_index('时间',drop=True)
delta_CPI['同比CPI变化'] = delta_CPI['同比CPI'].pct_change()
# delta_CPI['同比CPI变化'] = delta_CPI['同比CPI'].diff()
delta_CPI

,时间,同比CPI,同比CPI变化
0,2010-01-29,1.5300,NaN
1,2010-02-26,2.6953,0.761634
2,2010-03-31,2.3707,-0.120432
3,2010-04-30,2.8136,0.186822
4,2010-05-31,3.0613,0.088037
...,...,...,...
149,2022-06-30,2.5000,0.190476
150,2022-07-29,2.7000,0.080000
151,2022-08-31,2.5000,-0.074074
152,2022-09-30,2.8000,0.120000


In [110]:
monthly_ret = pd.read_excel('./B.1 - Choice数据/品种月收益率.xlsx')
monthly_ret

,时间,月收益率,品种
0,2010-01-29,0.954185,A
1,2010-02-26,0.994031,A
2,2010-03-31,1.015144,A
3,2010-04-30,1.004593,A
4,2010-05-31,0.960883,A
...,...,...,...
5293,2022-06-30,0.950891,sc
5294,2022-07-29,0.989018,sc
5295,2022-08-31,1.084394,sc
5296,2022-09-30,0.959870,sc


In [111]:
table = pd.merge(monthly_ret,delta_CPI,how='left',on='时间')
table = table.dropna()
table

,时间,月收益率,品种,同比CPI,同比CPI变化
1,2010-02-26,0.994031,A,2.6953,0.761634
2,2010-03-31,1.015144,A,2.3707,-0.120432
3,2010-04-30,1.004593,A,2.8136,0.186822
4,2010-05-31,0.960883,A,3.0613,0.088037
5,2010-06-30,0.997390,A,2.9499,-0.036390
...,...,...,...,...,...
5293,2022-06-30,0.950891,sc,2.5000,0.190476
5294,2022-07-29,0.989018,sc,2.7000,0.080000
5295,2022-08-31,1.084394,sc,2.5000,-0.074074
5296,2022-09-30,0.959870,sc,2.8000,0.120000


In [112]:
PZ_list = pd.read_excel('.//B.1 - Choice数据//future-factor_Choice版.xlsx')
PZ_list =  PZ_list['品种'].values.tolist()

In [113]:
# betas = pd.DataFrame(columns=['时间','品种','beta'])
for i in PZ_list:
    print(i)
    dff = table.loc[table['品种']==i]
    x = dff['同比CPI变化']
    y = dff['月收益率']
    if len(x)>36:
        results = PandasRollingOLS(x, y, 36)  # window 是滚动回归的自变量个数
        add = pd.DataFrame(columns=['时间','品种','beta'])
        add['beta'] = results.beta  # 每一步估计的斜率
        add['品种'] = i
        add['时间'] = dff.iloc[35:,0]
#         print(len(add))
        try:
            betas = betas.append(add)
        except:
            betas = add

RU
RR
RM
RB
nr
PB
P
OI
sc
PP
SF
Y
SN
SP
SR
SS
TA
UR
V
NI
SM
MA
EG
L
ZC
A
AG
AL
AP
AU
B
BU
C
CF
CJ
M
CS
EB
FG
FU
HC
I
J
JD
JM
CU
ZN


In [114]:
betas.to_excel('./A.7 - Beta因子/beta系数.xlsx',index=False)